In [25]:
import numpy as np
import pandas as pd

In [26]:
df = pd.read_csv('train.csv')

In [27]:
df.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [31]:
df['message'][2]

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [34]:
df['message'][5]

"Worth a read whether you do or don't believe in climate change https://t.co/ggLZVNYjun https://t.co/7AFE2mAH8j"

In [5]:
df['sentiment'].value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

In [6]:
##checking null values
df.isnull().sum()

sentiment    0
message      0
tweetid      0
dtype: int64

In [7]:
blank_index = [i[0] for i in df.itertuples() if i[2].isspace()]

In [8]:
blank_index

[]

In [9]:
##data splitting
from sklearn.model_selection import train_test_split
X = df['message']
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC,SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score

In [19]:
txt1_clf = Pipeline([('vectorizer',TfidfVectorizer(stop_words='english',strip_accents='ascii')),('clf',LinearSVC())])
txt2_clf = Pipeline([('vectorizer',TfidfVectorizer(stop_words='english',strip_accents='ascii')),('clf',MultinomialNB())])
txt3_clf = Pipeline([('vectorizer',TfidfVectorizer(stop_words='english',strip_accents='ascii')),('clf',RandomForestClassifier(n_estimators=100))])
txt4_clf = Pipeline([('vectorizer',TfidfVectorizer(stop_words='english',strip_accents='ascii')),('clf',SVC(C=1.0,probability=True))])

In [20]:
def fit_all(lst,X,y):
    lis = []
    for pip in lst:
        pip.fit(X,y)
        lis.append(pip)
    return lis


In [21]:
trained=fit_all([txt1_clf,txt2_clf,txt3_clf,txt4_clf],X=X_train,y=y_train)

C:\Users\1437795\AppData\Local\Continuum\anaconda3\envs\nlp_course\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [22]:
trained

[Pipeline(memory=None,
      steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=...ax_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0))]),
 Pipeline(memory=None,
      steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=...rue,
         vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
 Pipeline(memory=None,
      steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',


In [23]:
##LinearSV
pip1_prediction = trained[0].predict(X_test)
##Naive
pip2_prediction = trained[1].predict(X_test)
##Random forest
pip3_prediction = trained[2].predict(X_test)
##SVC
pip4_prediction = trained[3].predict(X_test)

In [24]:
print('LinearSV')
print(classification_report(y_test,pip1_prediction))
print('\n\n')
print('Naive')
print(classification_report(y_test,pip2_prediction))
print('\n\n')
print('Random forest')
print(classification_report(y_test,pip3_prediction))
print('\n\n')
print('SVC')
print(classification_report(y_test,pip4_prediction))

LinearSV
              precision    recall  f1-score   support

          -1       0.76      0.48      0.59       428
           0       0.60      0.42      0.50       737
           1       0.78      0.84      0.81      2861
           2       0.70      0.78      0.74      1195

   micro avg       0.74      0.74      0.74      5221
   macro avg       0.71      0.63      0.66      5221
weighted avg       0.73      0.74      0.73      5221




Naive
              precision    recall  f1-score   support

          -1       1.00      0.01      0.03       428
           0       1.00      0.06      0.11       737
           1       0.60      0.99      0.75      2861
           2       0.90      0.33      0.48      1195

   micro avg       0.63      0.63      0.63      5221
   macro avg       0.87      0.35      0.34      5221
weighted avg       0.76      0.63      0.54      5221




Random forest
              precision    recall  f1-score   support

          -1       0.85      0.22      0

C:\Users\1437795\AppData\Local\Continuum\anaconda3\envs\nlp_course\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
##testing Accuracy score
print('LinearSV')
print(accuracy_score(y_test,pip1_prediction))
print('\n\n')
print('Naive')
print(accuracy_score(y_test,pip2_prediction))
print('\n\n')
print('Random forest')
print(accuracy_score(y_test,pip3_prediction))

LinearSV
0.748898678414097



Naive
0.6198046351273703



Random forest
0.7077188278107642


In [19]:
##LinearSV
pip1_predictiont = trained[0].predict(X_train)
##Naive
pip2_predictiont = trained[1].predict(X_train)
##Random forest
pip3_predictiont = trained[2].predict(X_train)

In [20]:
##training Accuracy score
print('LinearSV')
print(accuracy_score(y_train,pip1_predictiont))
print('\n\n')
print('Naive')
print(accuracy_score(y_train,pip2_predictiont))
print('\n\n')
print('Random forest')
print(accuracy_score(y_train,pip3_predictiont))

LinearSV
0.9939611247405171



Naive
0.6755048122287224



Random forest
0.9999056425740706


In [21]:
df2 = pd.read_csv('test.csv')

In [23]:
df2.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928
